# 14_TreeQSM-Zylinder_to_Skeleton
In diesem Jupyter Notebook werden die resultierenden Zylinder aus TreeQSM zu einem Skelett verarbeitet

### 1. Zylinder zu PLY
(Darstellung der edges funktioniert nicht)

In [9]:
import pandas as pd
import numpy as np

def write_ply_segments(filename, starts, ends):
    vertices = np.vstack([starts, ends])
    n_vertices = vertices.shape[0]
    n_edges = starts.shape[0]
    edges = np.column_stack([np.arange(n_edges), np.arange(n_edges) + n_edges])

    with open(filename, 'w') as f:
        f.write('ply\n')
        f.write('format ascii 1.0\n')
        f.write(f'element vertex {n_vertices}\n')
        f.write('property float x\n')
        f.write('property float y\n')
        f.write('property float z\n')
        f.write(f'element edge {n_edges}\n')
        f.write('property int vertex1\n')
        f.write('property int vertex2\n')
        f.write('end_header\n')
        for v in vertices:
            f.write(f'{v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n')
        for e in edges:
            f.write(f'{e[0]} {e[1]}\n')

# --------------------------------------------------------
# Pfade manuell anpassen
file_pointcloud = r'140_BaseData\pointcloud\20250324_DJIPhantom4ProV2_Kirsche_clipped_cleaned.txt'
file_cylinders = r'140_BaseData\cylinder_tree_t1_m1_D0.08_DA0.1_DI0.02.txt'
output_file = r'G:/53_Skelettierung/20250324_DJIPhantom4ProV2_Kirsche_Skelett_test_2.ply'

# Schritt 1: Landeskoordinaten-Mittelwert berechnen
df_points = pd.read_csv(file_pointcloud, delimiter=';', skiprows=1, header=None)
P_mean = df_points.iloc[:, :3].mean().to_numpy()

# Schritt 2: Zylinderdatei mit 17 Spalten einlesen
columns = [
    'radius', 'length',
    'X', 'Y', 'Z',
    'dX', 'dY', 'dZ',
    'ParentID', 'SegmentID', 'Branch',
    'BranchOrder', 'PositionInBranch',
    'mad', 'SurfCov', 'added', 'UnmodRadius'
]
T = pd.read_csv(file_cylinders, delimiter='\t', names=columns, skiprows=1)

# Schritt 3: Start- und Endpunkte berechnen
starts = T[['X', 'Y', 'Z']].to_numpy()
directions = T[['dX', 'dY', 'dZ']].to_numpy()
lengths = T['length'].to_numpy().reshape(-1, 1)
ends = starts + directions * lengths


# Schritt 4: Rückverschiebung in LV95-Koordinaten
starts += P_mean
ends += P_mean

# Schritt 5: Skelett als PLY-Datei exportieren
write_ply_segments(output_file, starts, ends)


### 2. Zylinder zu OBJ
(Darstellung der edges funktioniert nicht)

In [12]:
import pandas as pd
import numpy as np

def write_obj_segments(filename, starts, ends):
    vertices = np.vstack([starts, ends])
    with open(filename, 'w') as f:
        for v in vertices:
            f.write(f'v {v[0]:.6f} {v[1]:.6f} {v[2]:.6f}\n')
        for i in range(starts.shape[0]):
            f.write(f'l {i + 1} {i + 1 + starts.shape[0]}\n')

# -------------------- EINSTELLUNGEN --------------------
file_pointcloud = r'140_BaseData\pointcloud\20250324_DJIPhantom4ProV2_Kirsche_clipped_cleaned.txt'
file_cylinders = r'140_BaseData\cylinder_tree_t1_m1_D0.08_DA0.1_DI0.02.txt'
output_file = r'G:/53_Skelettierung/20250324_DJIPhantom4ProV2_Kirsche_Skelett_testOBJ.obj'

# -------------------- PUNKTE LADEN ---------------------
df_points = pd.read_csv(file_pointcloud, delimiter=';', skiprows=1, header=None)
P_mean = df_points.iloc[:, :3].mean().to_numpy()

# -------------------- ZYLINDERDATEN ---------------------
columns = [
    'radius', 'length',
    'X', 'Y', 'Z',
    'dX', 'dY', 'dZ',
    'ParentID', 'SegmentID', 'Branch',
    'BranchOrder', 'PositionInBranch',
    'mad', 'SurfCov', 'added', 'UnmodRadius'
]
T = pd.read_csv(file_cylinders, delimiter='\t', names=columns, skiprows=1)

# -------------------- START/ENDPUNKTE --------------------
starts = T[['X', 'Y', 'Z']].to_numpy()
directions = T[['dX', 'dY', 'dZ']].to_numpy()
lengths = T['length'].to_numpy().reshape(-1, 1)
ends = starts + directions * lengths

# -------------------- RÜCKTRANSFORMATION --------------------
starts += P_mean
ends += P_mean

# -------------------- EXPORT ALS OBJ --------------------
write_obj_segments(output_file, starts, ends)
